심층 신경망이 학습한 모든 변환을 수치 데이터 텐서에 적용하는 **Tensor Operation** (텐서 연산)

첫번째 예제에서 Dense 층을 쌓아서 신경망을 만들었음

```python
keras.layers.Dense(512, activation = 'relu')
```

이 층은 2D Tensor를 입력받고 새로운 표현인 또 다른 2D Tensor를 반환하는 함수로 해석할 수 있음.

구체적으로 보면:

```python
output = relut(dot(W, input) + b)
```

3가지 텐서 연산이 있음
* 입력 텐서와 텐서 `W`사이의 dot product
* dot product의 결과인 2D 텐서와 b 사의 덧셈
* 마지막으로 `relu` 연산

**relu(x) = max(x, 0)**

### 2.3.1 원소별 연산

* `relu` 함수와 덧셈은 원소별 연산 (**element-wise operation**)
* 텐서에 있는 각 원소에 독립적으로 적용

In [1]:
def naive_relu(x):
    ## x is 2D numpy array
    assert len(x.shape) == 2 
    
    ## 입력 텐서 자체를 바꾸지 않도록 복사합니다.
    x = x.copy()
    
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i,j] = max(x[i,j], 0)
            
    return x

In [3]:
def naive_add(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
            
    return x

In [4]:
def naive_subtract(x, y):
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] -= y[i, j]
            
    return x

같은 원리로 원소별 곱셈, 뺄셈 등을 할 수 있음

### 2.3.2 브로드캐스팅

덧셈 구현인 `naive_add`는 동일한 크기의 2D Tensor의 덧셈을 지원함. 하지만 Dense층에서는 2D Tensor와 Vector를 더하였음. 크기가 다른 두 Tensor를 더했을때 **무슨 일이 일어날까?**

* 모호하지 않고 실행 가능하다면 *작은* 텐서가 *큰* 텐서의 크기에 맞추어 **Broadcasting**됨
    1. 큰 텐서의 ndim에 맞도록 작은 텐서에 축이 추가 됨 (**Broadcasting Axis**)
    2. 작은 텐서가 새 축을 따라서 큰 텐서의 크기에 맞도록 반복됨

예시:

* X의 크기는 (32, 10)이고 y의 크기는 (10,)라고 가정
* y에 비어있는 첫번째 축을 추가하여 크기를 (1, 10)로 만듦
* 그런 다음 y를 이 축에 32번 반복하면 텐서 Y의 크기는 (32, 10)이 됨
* Y[i, :] == y for i in range(0, 32)
* X와 Y는 크기가 같으므로 더할 수 있음

구현 입장에서는 새로운 텐서가 만들어지면 매우 **비효율적**이므로 어떤 2D Tensor로 만들어지지 않음. 반복된 연산은 가상적.

In [7]:
def naive_add_matrix_and_vector(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[j]
            
    return x

### 2.3.3 Tensor Product

In [8]:
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z = 0
    for i in range(x.shape[0]):
        z += x[i] * y[i]
        
    return z

두 벡터의 점곱은 스칼라가 되므로 원소 개수가 같은 벡터끼리 점곱이 가능

In [9]:
import numpy as np

def naive_matrix_vector_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 1
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            z[i] += x[i,j] * y[j]
    return z

행렬-벡터 점곱과 벡터-벡터 점곱 사이의 관계를 부각하기 위해 앞에서 만든 함수를 재활용 함

In [10]:
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z

두 텐서 중 하나라도 ndim이 1보다 크면 dot product의 교환 법칙이 성립되지 않음
* 벡터와 벡터간의 점곱은 교환 법칙이 성립하지만 행렬-벡터, 행렬-행렬은 성립하지 않음)
* 즉, dot(x,y)와 dot(y,x)는 다름

In [11]:
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            col_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, col_y)
            
    return z

![](./images/2-3-3-dot.png)

* x의 행 벡터와 y의 열 벡터가 같은 크기여야 함
* 즉, x의 너비와 y의 높이는 동일함

(a, b, c, d) . (d, ) -> (a, b, c)

(a, b, c, d) . (d, e) -> (a, b, c, e)

### 2.3.4 텐서 크기 변환 (Tensor Reshaping)

텐서의 크기를 변환한다는 것은 특정 크기에 맞게 열과 행을 *재배열*한다는 뜻

In [12]:
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])

print(x.shape)

(3, 2)


In [13]:
x = x.reshape((6, 1))
x

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [14]:
x = x.reshape((2, 3))
x

array([[0., 1., 2.],
       [3., 4., 5.]])

자주 사용하는 특별한 크기 변환은 **Transpose** (전치)라고 불림. 행렬의 전치는 행과 열을 바꾸는 것. 즉, `x[i, :]`이 `x[:, i]`가 됨

In [15]:
x = np.zeros((300, 20))
x = np.transpose(x)
print(x.shape)

(20, 300)
